# 图数据库：Neo4j

知识图谱需要专门的数据库进行存储和查询，这类数据库被称为图数据库 (Graph Database)。

与传统的关系型数据库（如 MySQL）相比，图数据库的优势在于其对“关系”的查询性能。

对于需要进行多层关系遍历的复杂查询（例如，查询“我朋友的朋友”），图数据库的效果远超关系型数据库。

Neo4j 就是目前比较流行的一款开源图数据库。

## 核心概念

- 节点 (Node)：节点是图中的基本数据单元，用于表示现实世界中的实体.

    - 例如一个人、一家公司、一本书或一个账户。
    - 在关系型数据库中，节点可以类比为表中的一行。

- 标签 (Label)：用于为节点分类或打上“类型”标记。

    - 一个节点可以拥有一个或多个标签。
    - 例如，一个节点可以同时拥有 :Person 和 :Author 两个标签，表示这个人既是一个普通人，也是一位作者。

- 关系 (Relationship)：这是图数据库的精髓所在，它以一种富有表现力的方式连接两个节点，并明确地定义了它们之间的联系。

    - 每个关系都具有以下特点：

      - 有方向：关系总是从一个“起始节点”指向一个“结束节点”。
      - 有类型：每个关系都必须有一个类型（例如 :FRIENDS_WITH, :PURCHASED），用来描述连接的性质。
      - 有属性：和节点一样，关系也可以存储属性，例如，一个 :PURCHASED 关系可以有一个 date 属性来记录购买日期。

- 属性 (Property)：属性是以键值对（Key-Value）形式存储在节点和关系上的详细信息。

    - 键是字符串，值可以是各种基本数据类型（如字符串、数字、布尔值）或它们的数组。


![](images/22_1.png)

`Alice` 和 `Bob` 是 节点，`:Person` 是 标签，`{name: 'Alice'}` 是属性。

`KNOWS` 则是连接它们的 关系 类型，而 `{since: 2020}` 是这段关系上的 属性。

## 查询语言：Cypher

Cypher 是 Neo4j 的声明式图形查询语言，语法灵感来源于 SQL，但针对图的特性进行了优化。

通过 Cypher，可以用一种直观且高效的方式来查询和操作图数据。

MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie {title: 'The Matrix'})

RETURN actor.name

查找在电影《黑客帝国》(The Matrix) 中出演过的所有演员。

官方的 Cypher 语法速查表：https://neo4j.com/docs/cypher-refcard/4.4/

汇总了常用的命令、操作符和语法结构，可供读者快速查阅。

##  安装

### Neo4j Desktop（推荐用于本地学习）

#### 安装

1. 首先访问 Neo4j 官网进行下载：https://neo4j.com/download/

2. 点击网页中的 `download` 按钮，跳转到一个注册表单。

3. 填写任意信息后，点击 `Download Desktop` 按钮，开始下载 Neo4j Desktop 安装包。

4. 下载完成后，双击安装文件，程序会自动进行安装。

5. 安装完成后首次启动，会看到许可协议界面，点击 “Continue” 即可完成最后的设置。

如何更改安装路径请看：

https://zhuanlan.zhihu.com/p/1935104156433121644

https://jishuzhan.net/article/1987823225158565889

### Docker（推荐用于服务器部署与跨平台开发）

安装只需一行命令即可完成拉取镜像和启动容器。

docker run \
    --name my-neo4j \
    -p 7474:7474 -p 7687:7687 \
    -d \
    -v $HOME/neo4j/data:/data \
    -v $HOME/neo4j/logs:/logs \
    --env NEO4J_AUTH=neo4j/password \
    neo4j:latest

- -p 7474:7474: 将容器的 HTTP 端口映射到本机，用于浏览器访问。

- -p 7687:7687: 将容器的 Bolt 驱动端口映射到本机，用于代码连接。

- -v $HOME/neo4j/data:/data: 将数据目录挂载到本机，确保数据持久化。

- --env NEO4J_AUTH=neo4j/password: 设置数据库的初始用户名和密码（此处为 neo4j/password）。

- neo4j:latest: 使用最新的官方镜像。

## Neo4j 基本使用

### 创建并连接数据库

1. 创建实例：打开 Neo4j Desktop，在 “Local instances” 页面点击 “Create instance” 按钮。

2. 配置实例：在弹出的窗口中，为实例命名（例如 base nlp），选择所需的 Neo4j 版本，并为默认用户 neo4j 设置一个能记住的密码。完成后点击 “Create”。

    - Neo4j 版本会有一个已经下载好的，在 `\resources\offline\dbmss` 中，选择其他版本需要再次下载，时间有些长。

3. 启动与连接：实例创建后会自动启动，状态显示为 “RUNNING”。
   
    - 可以通过浏览器直接访问 http://127.0.0.1:7474 来打开 Neo4j Browser。

### 增删查改

使用 Cypher 语言来进行增删查改操作。

#### 创建节点

创建节点的基本语法是 CREATE (变量:标签 {属性: 值})。

- 变量 (Variable)：一个临时名称，用于在同一条语句中引用该节点。

- 标签 (Label)：用于对节点进行分类。

- 属性 (Properties)：一个包含键值对的 map/字典，用于描述节点的具体信息。

最基础的创建语句包含一个临时变量、一个标签和一组属性。

CREATE (pork:Ingredient {name:'猪肉', category:'肉类', origin:'杭州'});

![](images/22_2.png)

如果在创建后不需要立刻使用这个节点（例如，在同一查询中创建关系），可以省略临时变量名，这样语法更简洁。

CREATE (:Ingredient {name:'土豆', category:'蔬菜', origin:'北京'});

![](images/22_3.png)


还可以在创建节点后，使用 RETURN 子句立即将其返回。这对于调试或确认节点是否按预期创建非常有用。

CREATE (n:Ingredient {name:'鸡蛋'}) RETURN n;

![](images/22_4.png)


执行上述三条命令后，数据库中就创建了三个 Ingredient 类型的节点。

能够通过 Neo4j Browser 的可视化界面直观地看到这些新创建的数据。

点击左侧 `Ingredients` 按钮即可查看，或输入 MATCH (n:Ingredient) RETURN n LIMIT 25;

![](images/22_5.png)


点击节点便可以在右侧看到其详细的属性信息。

- id 字段：Neo4j 为每个节点自动生成的内部唯一标识符。

- Key-Value 结构：“Key” 和 “Value” 两列展示了节点属性是以键值对的形式存储的。

- 自定义属性：name、category、origin 三个字段的值与前面 CREATE 语句中设定的值完全一致。

![](images/22_6.png)

#### 创建关系

关系的创建通常需要先指定关系两端的节点，然后用 -[变量:类型 {属性}]-> 来定义关系。

- 关系必须有 方向 和 类型 (Type)。

- 小括号 () 用于表示节点，中括号 [] 用于表示关系。

在实际应用中，常常需要一次性创建多个节点以及它们之间的关系。

CREATE 语句支持通过逗号分隔，在一个查询中完成复杂图谱的构建。

- 创建 4 个 Ingredient 节点：猪里脊、木耳、胡萝卜、青椒。

- 创建 2 个 Dish 节点：鱼香肉丝、木须肉。

- 创建 5 条 包含 关系：从菜品指向食材。

- 创建 5 条 被用于 关系：从食材指向菜品。

这样既可以方便地查询“一道菜包含哪些食材”，也可以高效地反向查询“一种食材被用在了哪些菜里”。

CREATE
	// 创建食材节点
	(rousi:Ingredient {name:'猪里脊'}),
	(muer:Ingredient {name:'木耳'}),
	(huluobo:Ingredient {name:'胡萝卜'}),
	(qingjiao:Ingredient {name:'青椒'}),
	// 创建菜品节点
	(d1:Dish {name:'鱼香肉丝', cuisine:'川菜'}),
	(d2:Dish {name:'木须肉', cuisine:'鲁菜'}),
	// 创建关系
	(d1)-[:包含 {amount:'250g'}]->(rousi), (d1)-[:包含]->(muer), (d1)-[:包含]->(huluobo),
	(d2)-[:包含 {amount:'150g'}]->(rousi), (d2)-[:包含]->(muer),
	// 创建双向关系
	(rousi)-[:被用于]->(d1), (muer)-[:被用于]->(d1), (huluobo)-[:被用于]->(d1),
	(rousi)-[:被用于]->(d2), (muer)-[:被用于]->(d2);

执行 MATCH p=()-[:包含]->() RETURN p LIMIT 25; 查询可以可视化展示所有“包含”关系。

![](images/22_7.png)

### 查询

MATCH 是 Cypher 中用于查询图数据的命令，它允许你描述你想要寻找的节点和关系的模式。

#### 基本查询

匹配并返回图中的任意节点，可以使用 LIMIT 关键字限制返回数量。

// 匹配并返回图中的任意 25 个节点
MATCH (n)
RETURN n
LIMIT 25;

![](images/22_8.png)

根据标签和属性进行精确匹配。

// 匹配所有标签为 Ingredient，且名字为'猪里脊'的节点
MATCH (n:Ingredient {name:'猪里脊'}) RETURN n;

![](images/22_9.png)

#### 条件查询

WHERE 子句提供了更灵活的过滤能力，可以对节点的属性进行复杂的逻辑判断。

查询名字是'猪里脊'或'鸡蛋'的 Ingredient 节点。

MATCH (n:Ingredient)
WHERE n.name IN ['猪里脊','鸡蛋']
RETURN n;

![](images/22_10.png)

也可以使用 AND、OR 等关键字构建复合查询条件。

MATCH (n:Ingredient)
WHERE n.name IN ['猪肉', '猪里脊', '鸡蛋'] AND n.category = '肉类'
RETURN n;

![](images/22_11.png)

#### 返回指定属性

默认情况下，RETURN n 会返回整个节点对象。

也可以只返回节点的特定属性，并使用 AS 为返回的列起别名，使结果更具可读性。

MATCH (n:Ingredient)
WHERE n.name IN ['猪里脊','鸡蛋']
RETURN n.name AS 食材名称, n.category AS 类别;

![](images/22_12.png)

#### 关联查询

图数据库最强大的地方在于对关系的查询。

例如，可以一次性查询“鱼香肉丝”和“木须肉”分别包含了哪些食材。

其中，`collect()` 是一个聚合函数，可以将匹配到的多个同类结果（这里是食材名称 i.name）收集到一个列表中。

MATCH (d:Dish)-[:包含]->(i:Ingredient)
WHERE d.name IN ['鱼香肉丝', '木须肉']
RETURN d.name AS 菜品, collect(i.name) AS 食材列表;

![](images/22_13.png)

#### 查询并创建 (MATCH + CREATE)

通过组合使用 MATCH 和 CREATE 实现先找到图中已经存在的节点，然后为它们添加新的关系。

例如，我们已经创建了“鱼香肉丝”和“猪里脊”，现在想为它们添加一条“主要食材”的关系。

其中，MERGE 避免重复的关系。

MATCH
    (d:Dish {name:'鱼香肉丝'}),
    (i:Ingredient {name:'猪里脊'})
MERGE
    (d)-[r:主要食材]->(i)
RETURN d, i, r;

![](./images/22_14.png)


#### 排序 (ORDER BY)

使用 ORDER BY 子句对返回的结果进行排序。

默认是升序 (ASC)，也可以指定为降序 (DESC)。

// 查询所有食材，并按名称升序排序
MATCH (i:Ingredient)
RETURN i.name, i.category
ORDER BY i.name ASC;

![](images/22_15.png)

### 更新 (SET & MERGE)

#### 更新属性 (SET)

SET 语句用于修改或添加节点/关系的属性。

必须和 MATCH 配合使用，先找到要更新的实体，再进行修改。

MATCH (i:Ingredient {name:'猪肉'})
SET
    i.is_frozen = true,
    i.origin = '金华'
RETURN i;

![](images/22_16.png)

#### 插入或更新 (MERGE)

MERGE 会根据你提供的模式在图中查找，

如果找到匹配项，则执行 ON MATCH 部分；

如果未找到，则执行 ON CREATE 部分

`coalesce(property, defaultValue)` 函数会检查属性 property 是否存在，如果存在则返回其值，否则返回 defaultValue。

// 查找名为'大蒜'的 Ingredient 节点
MERGE (n:Ingredient {name: '大蒜'})
// 如果不存在，则创建该节点，并设置创建时间和初始库存
ON CREATE SET
    n.created = timestamp(),
    n.stock = 100
// 如果已存在，则更新其库存、访问次数和访问时间
ON MATCH SET
  n.stock = coalesce(n.stock, 0) - 1,
  n.counter = coalesce(n.counter, 0) + 1,
  n.accessTime = timestamp()
RETURN n;

![](images/22_17.png)

### 删除 (DELETE & REMOVE)

#### 删除属性 (REMOVE)

REMOVE 用于移除节点或关系上的某个属性。

先用 MATCH 找到名为“大蒜”的节点，然后移除由 MERGE 命令在创建它时添加的 created 属性。

MATCH (i:Ingredient {name:'大蒜'})
REMOVE i.created
RETURN i;

####  删除节点和关系 (DELETE)

DELETE 用于删除节点和关系。

**Neo4j 不允许直接删除一个还存在关联关系的节点。你必须先删除关系，才能删除节点。**

MATCH (i:Ingredient {name:'大蒜'})-[r]-() // 匹配与'大蒜'相连的任意关系
DELETE r, i; // 先删除关系 r，再删除节点 i

或者使用 DETACH DELETE，它会自动删除指定节点以及所有与它直接相连的关系。

// 使用 DETACH DELETE (推荐)
MATCH (i:Ingredient {name:'大蒜'})
DETACH DELETE i;

![](images/22_18.png)

还可以通过节点的内部 ID 进行精确查找和删除。

每个节点都有一个由 Neo4j 自动分配的唯一 ID，可以通过 id() 函数获取。

// 假设我们通过查询得知“大蒜”的 ID 为 5
MATCH (i:Ingredient)
WHERE id(i) = 5
DETACH DELETE i;

#### 清空数据库

删除数据库中的所有节点和关系

// 匹配所有节点 n
MATCH (n)
// 强制删除节点 n 及其所有关系
DETACH DELETE n;

![](images/22_20.png)


#### 软删除

软删除并非真的将数据移除，而是通过 SET 命令为其添加一个状态属性，将其标记为“已删除”或“不活跃”。

在后续的查询中，只需要增加一个 WHERE i.is_active = true 的过滤条件，就能只使用那些“活跃”的数据，

而被软删除的数据依然保留在数据库中，以备审计或恢复。

// 将“木耳”标记为不活跃
MATCH (i:Ingredient {name:'木耳'})
SET i.is_active = false;

![](images/22_19.png)